In [1]:
%set_env SPARK_HOME=/opt/spark
import os
import findspark
import pandas as pd
os.environ['PYSPARK_SUBMIT_ARGS'] = """\
--master spark://10.62.42.237:7077 
--packages datastax:spark-cassandra-connector:2.3.1-s_2.11,anguenot:pyspark-cassandra:0.9.0
--driver-memory 8g
--executor-memory 3g
--conf spark.local.dir=/var/spark
--conf spark.network.timeout=100000s
--conf spark.executor.heartbeatInterval=10000s
--conf spark.shuffle.registration.timeout=10000s
pyspark-shell
"""


import pyspark
from pyspark import SparkContext, SparkConf
findspark.init()
sc = pyspark.SparkContext(appName="TimeOfDay")

sc._conf.set("spark.sql.execution.arrow.enabled", "true")
sc._conf.set('spark.driver.maxResultSize', '6g')
# sc._conf.set('spark.driver.cores', '4')
# sc._conf.set('spark.executor.cores', '4')
# sc._conf.set('spark.driver.memory', '24g')
# sc._conf.set('spark.executor.memory', '24g')
# sc._conf.set("spark.cores.max", 8)
sc._conf.set("spark.local.dir", "/var/spark")

# sc._conf.set("spark.network.timeout", "1000000s")
# sc._conf.set("spark.executor.heartbeatInterval", "1000000s")
#'10.224.12.32'

sc._conf.set("spark.cassandra.connection.host", "10.62.1.118")
sc._conf.set("spark.cassandra.auth.username", "mlprw")
sc._conf.set("spark.cassandra.auth.password", "q4RgwD$wK7*z")

sc._conf.getAll()

env: SPARK_HOME=/opt/spark


[('spark.master', 'spark://10.62.42.237:7077'),
 ('spark.driver.port', '22161'),
 ('spark.deploy.spreadOut', 'true'),
 ('spark.sql.execution.arrow.enabled', 'true'),
 ('spark.local.dir', '/var/spark'),
 ('spark.files',
  'file:///root/.ivy2/jars/datastax_spark-cassandra-connector-2.3.1-s_2.11.jar,file:///root/.ivy2/jars/anguenot_pyspark-cassandra-0.9.0.jar,file:///root/.ivy2/jars/com.twitter_jsr166e-1.1.0.jar,file:///root/.ivy2/jars/org.joda_joda-convert-1.2.jar,file:///root/.ivy2/jars/io.netty_netty-all-4.0.33.Final.jar,file:///root/.ivy2/jars/commons-beanutils_commons-beanutils-1.9.3.jar,file:///root/.ivy2/jars/joda-time_joda-time-2.3.jar,file:///root/.ivy2/jars/org.scala-lang_scala-reflect-2.11.8.jar,file:///root/.ivy2/jars/commons-collections_commons-collections-3.2.2.jar,file:///root/.ivy2/jars/com.datastax.spark_spark-cassandra-connector_2.11-2.3.0.jar,file:///root/.ivy2/jars/net.razorvine_pyrolite-4.10.jar,file:///root/.ivy2/jars/net.razorvine_serpent-1.12.jar'),
 ('spark.cassan

In [4]:
import pyspark
from pyspark.sql import SparkSession, SQLContext
WORK_DIR = '/var/spark/ml_files/'
spark = SparkSession.builder.\
    master("spark://10.62.42.237:7077").\
    appName("TimeOfDay").\
    config(conf=sc._conf).\
    getOrCreate()
spark

sqlContext = SQLContext(sc)
sqlContext

In [35]:
import time

import pandas as pd


from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit, lower
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import pyspark_cassandra
        
import calendar, datetime
from datetime import timedelta


def convert_status(value):
    if value == 'Completed' or value == 'Reversed':
        return 1
    else:
        return 0

def _map_to_pandas(rdds):
    """ Needs to be here due to pickling issues """
    return [pd.DataFrame(list(rdds))]

def toPandas(df, n_partitions=None):
    """
    Returns the contents of `df` as a local `pandas.DataFrame` in a speedy fashion. The DataFrame is
    repartitioned if `n_partitions` is passed.
    :param df:              pyspark.sql.DataFrame
    :param n_partitions:    int or None
    :return:                pandas.DataFrame
    """
    if n_partitions is not None: df = df.repartition(n_partitions)
    df_pand = df.rdd.mapPartitions(_map_to_pandas).collect()
    df_pand = pd.concat(df_pand)
    df_pand.columns = df.columns
    return df_pand
    
udfValueToStatus = udf(convert_status, IntegerType())  
createDDL = """CREATE OR REPLACE TEMPORARY VIEW cpg_transaction_history
     USING org.apache.spark.sql.cassandra
     OPTIONS (
     table "cpg_transaction_history",
     keyspace "dev_sarsks",
     cluster "MLP Cluster",
     pushdown "true")"""
spark.sql(createDDL)

# df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date > '2018-08-12' ")
df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date > '2018-08-11' and  received_date < '2018-10-01' ")
# df = spark.sql("SELECT * FROM cpg_transaction_history")

sdf =  df\
    .withColumn("success", udfValueToStatus("new_status"))\
    .withColumn("day_of_month", df["day_of_month"].cast(IntegerType()))\
    .withColumn("transaction_hour", df["transaction_hour"].cast(IntegerType()))\
    .withColumn("day_of_week", lower(col("day_of_week")))\
    .withColumn("payment_amount_usd", df["payment_amount_usd"].cast(FloatType()))\
    .cache()

print(sdf.schema)

pdf_all =sdf.toPandas() 
# pdf_all = toPandas(sdf, 4)  
pdf_all.shape

StructType(List(StructField(received_date,DateType,true),StructField(site_id,StringType,true),StructField(subscription_id,StringType,true),StructField(transaction_id,StringType,true),StructField(transaction_date,TimestampType,true),StructField(added_expiry_years,StringType,true),StructField(bank_code,StringType,true),StructField(bank_name,StringType,true),StructField(billing_country,StringType,true),StructField(bin,StringType,true),StructField(card_brand,StringType,true),StructField(card_category,StringType,true),StructField(card_class,StringType,true),StructField(card_usage,StringType,true),StructField(cc_expiration_date,StringType,true),StructField(cid,StringType,true),StructField(date_increment,StringType,true),StructField(day_of_month,IntegerType,true),StructField(day_of_week,StringType,true),StructField(event_uuid,StringType,true),StructField(exp_setting_id,StringType,true),StructField(funding_source,StringType,true),StructField(issuer_country,StringType,true),StructField(merchant

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 11540)
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.6/site-packages/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.6/site-packages/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/opt/anaconda3/lib/python3.6/site-packages/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.6/socketserver.py", line 317, in _handle_reques

(1842022, 46)

In [4]:
sc.stop()

In [36]:
pdf_all[pdf_all['bin'] == ''].head(20)

received_date   site_id subscription_id transaction_id  \
25765     2018-08-23   adbehme     11985934101     7254144073   
45411     2018-08-23     avast      3847001001     7255474353   
45490     2018-08-23     avast      5883712801     7254787823   
45644     2018-08-23     avast      9154506701     7254481703   
75927     2018-08-23    kasper     11125771801     7253903403   
75928     2018-08-23    kasper     11125866401     7253903753   
76140     2018-08-23  kasperde     10012023001     7254319633   
76141     2018-08-23  kasperde     10124107501     7257131913   
76393     2018-08-23  kasperde      2020941109     7256304713   
76784     2018-08-23  kasperde      3534456901     7257132063   
76927     2018-08-23  kasperde      3682262901     7254393093   
76928     2018-08-23  kasperde      4596972701     7257943913   
77204     2018-08-23  kasperde      9611860101     7256576233   
86057     2018-08-23    tmapac      1468809909     7253478963   
86058     2018-08-23    tmapac      1468809909     7253478973   
86476     2018-08-23    tmemea       813630709     7254818973   
86477     2018-08-23    tmemea       813630709     7254819003   
86478     2018-08-23    tmemea       813630709     7254819013   
86556     2018-08-23  trendoem      1290336109     7253464913   
159044    2018-08-25     capac      1677499309     7265726053   

          transaction_date added_expiry_years         bank_code  \
25765  2018-08-23 02:06:09                               NON3DS   
45411  2018-08-23 08:45:56                       EXPORT_MANDATE   
45490  2018-08-23 07:16:34                       EXPORT_MANDATE   
45644  2018-08-23 04:45:26                       EXPORT_MANDATE   
75927  2018-08-23 01:34:24                       EXPORT_MANDATE   
75928  2018-08-23 01:34:25                       EXPORT_MANDATE   
76140  2018-08-23 02:46:27                       EXPORT_MANDATE   
76141  2018-08-23 10:46:12                       EXPORT_MANDATE   
76393  2018-08-23 09:15:36                       EXPORT_MANDATE   
76784  2018-08-23 10:46:12                       EXPORT_MANDATE   
76927  2018-08-23 03:46:38                       EXPORT_MANDATE   
76928  2018-08-23 12:17:05                       EXPORT_MANDATE   
77204  2018-08-23 09:46:02                       EXPORT_MANDATE   
86057  2018-08-23 00:29:19                  6            NON3DS   
86058  2018-08-23 00:29:20                  6            NON3DS   
86476  2018-08-23 07:45:47                  0            NON3DS   
86477  2018-08-23 07:45:47                  0            NON3DS   
86478  2018-08-23 07:45:47                  0  03F4D8767ACFA619   
86556  2018-08-23 00:22:55                  6  8136990D5DA16846   
159044 2018-08-25 00:27:14                  0            NON3DS   

             bank_name billing_country bin  card_brand  \
25765   CITIBANK, N.A.              AE      MasterCard   
45411                               DE                   
45490                               DE                   
45644                               AT                   
75927                               DE                   
75928                               DE                   
76140                               DE                   
76141                               DE                   
76393                               DE                   
76784                               DE                   
76927                               DE                   
76928                               DE                   
77204                               DE                   
86057                               TW                   
86058                               TW                   
86476                               SE                   
86477                               SE                   
86478                               SE                   
86556                               US                   
159044                              AU                   

              

In [11]:
# bin_profile.to_csv(WORK_DIR + 'bin_profile_time_of_day.csv')

In [28]:
pdf_all['site_id'].value_counts()  # ['avgstore', 'avast', 'kasperus', 'avastbr', 'tmamer', 'kasper', 'kasperde', 'tmapac', 'kasperuk', 'kaspersk', 'kasperla', 'tmecon', 'avgbr', 'tmemea']

avgstore    940583
avast       878513
adbehap     566007
adbehkr     263947
kasperus    227303
cppro       218324
adbehbr     182803
avastbr     123537
adbehme     120651
adbehru     109310
adbeheu      97773
tmamer       95203
kasper       78678
adbehil      59322
kasperde     49400
adbehtw      46599
tmapac       27821
kasperuk     25856
kasperla     17528
ciscoctg     15416
kaspersk     13178
logib2c      10153
tmecon        8854
avgbr         8845
adsk          8050
kasperbr      7837
kaspernl      7716
tmemea        7614
defendde      6972
nuanceus      6730
             ...  
kasucpuk       350
tmoemem        331
gdatasw        292
cybscrub       238
kasucpbr       213
nuancesm       202
defenduk       174
defendwo       161
omron          142
tmoemap        104
adskcct         87
ca              84
tmpseu          68
techsmit        59
kasucpkr        55
tmpsus          51
cajapan         51
kasperoe        48
caconsum        46
sapsubs         32
kasucpla        32
netapp      

In [51]:
"""Generates bin_profile based on payment_amount_usd"""
grouped = pdf_all['payment_amount_usd'].groupby(pdf_all['bin']).agg([('Mean','mean'),('Min','min'),('Max','max'),('StdDev','std'),'count']).reset_index()
bin_profile = grouped[grouped['count'] >= 20]
bin_profile = bin_profile[bin_profile['bin'] != '']
bin_profile

/opt/anaconda3/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:1351: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(self.var(ddof=ddof, **kwargs))


bin        Mean        Min          Max      StdDev  count
44     354035  171.332825  18.275478  1695.485107  366.251129     22
67     354102  336.178802  39.132500  1762.466553  472.294128     28
68     354103  186.487152   2.690000   849.919373  220.998566     41
99     357300   74.148285   8.070000   282.365570   86.108360     22
112    358277  128.297073  17.968691   567.421265  188.504211     28
117    358403   99.556259   7.960079  2628.920654  257.693237    122
121    358419  158.281662  18.364580   568.102966  176.780090     28
126    358622   58.165367  18.382374   397.269623   70.677002     26
139    358761  135.421799  18.364580   568.102966  201.559708     25
141    358787  140.534119   2.690000   564.731140  192.293930     25
142    360218   29.803118   9.990000   119.879997   19.950060     93
147    360857   29.015320   9.990000   179.880005   28.007475     94
169    364096   35.623180   2.490000   290.910004   82.585785     22
171    364172   29.482916   9.990000    84.989998   21.430880     24
179    370034   63.954193   2.610000   179.990005   42.710796     31
185    370276   71.082878   2.490000   230.279999   40.483807     59
188    370286   38.778343   8.662435   348.541229   72.832291     30
311    371240   60.800282   2.490000   228.000000   39.637989    106
312    371241   55.906963   2.490000   194.839996   37.172600     79
313    371242   69.738953   2.510000   160.860001   35.342236    105
314    371243   82.261429   2.490000   310.100006   56.520588     84
315    371244   66.783089   2.490000   194.839996   42.964409     81
316    371245   58.512466   2.490000   206.860001   43.050507     73
317    371246   53.983334   2.490000   201.880005   51.889771     97
318    371247   52.115376   2.490000   173.789993   38.634674     73
319    371248   54.200748   2.490000   179.990005   42.918842    120
320    371249   82.578888   2.490000   195.960007   65.284332     63
331    371270   52.293636   2.490000   193.630005   42.827965     55
332    371271   64.101273   2.490000   370.000000   50.027122    157
333    371272   56.426128   2.490000   169.589996   37.951401    173
...       ...         ...        ...          ...         ...    ...
32136  561243   60.824688   9.281184    92.668411   17.875465     90
32137  561256   61.804134  40.644382   140.832260   25.877365     38
32138  561257   51.771191   4.634784    86.875916   16.420404     61
32139  561258   58.349022   4.570944    93.500313   21.600145     72
32140  561259   52.291069   4.556580    94.060242   18.917419    121
32149  581766   56.243130   9.281184    70.140305   17.902605     25
32152  589283   11.345369   5.330467    70.790001    9.702742     88
32153  601100   67.920364   2.490000  1673.439941   43.012024   8202
32155  601120   64.717514   2.490000  2953.199951   78.398163   1835
32157  601129   67.486397   2.490000   179.990005   37.640755    269
32158  601130   73.042030   2.490000   174.229996   39.999207    143
32159  601139   55.909050   2.490000   408.640015   50.159977    274
32160  601142   68.520454  53.110001   114.699997   16.946890     22
32161  601149   64.538162   2.490000   379.619995   39.968616   1788
32163  601193   60.840939   2.490000   159.360001   40.992435     32
32164  601195   73.732399  14.990000   179.990005   30.809362     25
32165  601197   56.614578   4.250000   120.160004   27.170044     59
32167  602011   63.781967  41.067764    92.916382   11.959373     29
32172  639004    9.152911   5.594851    11.832900    2.702421     26
32183  655732   25.733829   4.990000    99.949997   18.529058     47
32187  670999   51.292561  22.725044    92.580429   19.788235     28
32190  675754   49.881794  31.999325    82.150002   12.722353     36
32191  675761   53.443577   9.280385    77.160004   13.256742    114
32193  676115   58.934620   4.618026    92.908386   24.532219     69
32198  676219   45.141514   4.545009    92.908386   24.742514     75
32202  676241   44.572414   4.536630    70.410263   26.299810     22
32208  6

In [38]:
bin_profile = pd.read_csv(WORK_DIR + 'bin_profile_time_of_day.csv')

In [39]:
txn_count_all = pdf_all.groupby('transaction_hour')[('site_id')].count().reset_index().rename(columns={'site_id':'total_count'})
txn_count_processor = pdf_all.groupby(['transaction_hour', 'payment_service_id'])[('site_id')].count().reset_index().rename(columns={'site_id':'processor_count'})
txn_count_all = pd.merge(txn_count_processor, txn_count_all, left_on='transaction_hour', right_on='transaction_hour', how='left')
txn_count_all

transaction_hour payment_service_id  processor_count  total_count
0                   0              adyen             7425       286964
1                   0      drwp-discover               11       286964
2                   0        drwp-dlocal             2902       286964
3                   0   drwp-dlocal-auto              902       286964
4                   0            drwp-fd             4782       286964
5                   0          firstdata            13606       286964
6                   0              litle             8591       286964
7                   0                mes            10980       286964
8                   0                nab             6907       286964
9                   0       netgiro-amex             3341       286964
10                  0        netgiro-bms            78782       286964
11                  0        netgiro-bnp             1843       286964
12                  0         netgiro-br              906       286964
13                  0        netgiro-jcb              208       286964
14                  0        netgiro-seb           122681       286964
15                  0            pagador            23093       286964
16                  0            slimpay                4       286964
17                  1              adyen            23631       683324
18                  1      drwp-discover               40       683324
19                  1        drwp-dlocal             3514       683324
20                  1   drwp-dlocal-auto             1485       683324
21                  1            drwp-fd             7994       683324
22                  1          firstdata            29455       683324
23                  1              litle            22130       683324
24                  1                mes            22818       683324
25                  1                nab            16160       683324
26                  1       netgiro-amex             4685       683324
27                  1        netgiro-bms           195379       683324
28                  1        netgiro-bnp             7916       683324
29                  1         netgiro-br             4277       683324
..                ...                ...              ...          ...
225                14              litle             6219        29934
226                14                mes             8480        29934
227                14       netgiro-amex               50        29934
228                14        netgiro-bms             2474        29934
229                14        netgiro-bnp                2        29934
230                14        netgiro-seb              486        29934
231                14         paymentech             1342        29934
232                15        drwp-dlocal                1        33603
233                15            drwp-fd                6        33603
234                15          firstdata            12317        33603
235                15              litle             6960        33603
236                15                mes             9580        33603
237                15       netgiro-amex               90        33603
238                15        netgiro-bms             2639        33603
239                15        netgiro-bnp                2        33603
240                15        netgiro-seb              451        33603
241                15         paymentech             1555        33603
242                15      paypalExpress                2        33603
243                16            drwp-fd                5        48311
244                16          firstdata            19888        48311
245                16              litle            11241        48311
246                16                mes            13993        48311
247                16                nab                1        48311
248                16       netgiro-amex                6        48311
249                16        

In [41]:
# site_ids = ['avgstore', 'avast', 'kasperus', 'avastbr', 'tmamer', 'kasper', 'kasperde', 'tmapac', 'kasperuk', 'kaspersk', 'kasperla', 'tmecon', 'avgbr', 'tmemea']
site_ids = ['avgstore', 'avast', 'kasperus', 'avastbr', 'tmamer', 'kasper', 'kasperde', 'tmapac', 'kasperuk', 'kaspersk', 'kasperla', 'tmecon', 'avgbr', 'tmemea']

pdf_f = pdf_all[pdf_all['site_id'].isin(site_ids)]

In [52]:
"""Merges the bin profile with the original data"""
pdf_all_bin_profile = pd.merge(pdf_f, bin_profile[['bin','Mean','Min','Max','StdDev']], left_on='bin', right_on='bin', how='left')
pdf_all_bin_profile.shape

(1069014, 50)

In [54]:
pdf_all_bin_profile_with_count = pd.merge(pdf_all_bin_profile, txn_count_all[['transaction_hour', 'payment_service_id','processor_count','total_count']], left_on=['transaction_hour', 'payment_service_id'], right_on=['transaction_hour', 'payment_service_id'], how='left')
# pdf_all_bin_profile_with_count = pdf_all_bin_profile_with_count.drop(['Unnamed: 0'], axis=1)
pdf_all_bin_profile_with_count.shape  #(2452800, 46), (876794, 49)

(1069014, 52)

In [55]:
pdf_all_bin_profile_with_count.head()

received_date site_id subscription_id transaction_id    transaction_date  \
0    2018-08-23   avast     10010341101     7253629653 2018-08-23 00:59:05   
1    2018-08-23   avast     10017086901     7253627313 2018-08-23 00:58:51   
2    2018-08-23   avast     10021676301     7258325943 2018-08-23 16:15:39   
3    2018-08-23   avast     10022230801     7253625563 2018-08-23 00:58:40   
4    2018-08-23   avast     10022230801     7253625783 2018-08-23 00:58:41   

  added_expiry_years         bank_code                              bank_name  \
0                               NON3DS                               ASB Bank   
1                  4  7AC38540376F98E6         COMMONWEALTH BANK OF AUSTRALIA   
2                     847B99784061F302  Bank of America, National Association   
3                               NON3DS                 Firstrand Bank Limited   
4                               NON3DS                 Firstrand Bank Limited   

  billing_country     bin  card_brand                          card_category  \
0              NZ  454860        Visa                           Visa Classic   
1              AU  535316  MasterCard  (Unembossed) Standard MasterCard Card   
2              US  481583        Visa                           Visa Classic   
3              ZA  422824        Visa                          Visa Business   
4              ZA  422824        Visa                          Visa Business   

  card_class                                 card_usage cc_expiration_date  \
0   Consumer  True credit (No PIN/Signature capability)               0721   
1   Consumer  True credit (No PIN/Signature capability)              11/21   
2   Consumer           Debit Hybrid (PIN and Signature)               0719   
3   Business  True credit (No PIN/Signature capability)               0319   
4   Business  True credit (No PIN/Signature capability)               0319   

                cid date_increment  day_of_month day_of_week  \
0                                             23    thursday   
1                                4            23    thursday   
2                                             23    thursday   
3                                             23    thursday   
4  RETRY_DECLINED.2                           23    thursday   

                             event_uuid exp_setting_id funding_source  \
0  8667914f-33dc-49dd-a147-8f1bed0d4af1                        Credit   
1  745d8732-b0ad-4c7c-a670-49f259470b62        3837631         Credit   
2  203cb185-c4e1-48e4-bcd4-9a5bdb8b8def                         Debit   
3  60687471-9170-4e3f-9f62-5ead60002ed6                        Credit   
4  2e6e8cbc-cd6f-433c-b622-5da03937b520                        Credit   

  issuer_country merchant_descriptor         merchant_number  \
0             NZ  DRI*AVAST Software  1411163460-NZD-pacific   
1             AU  DRI*AVAST Software                 I7A0010   
2             US  DRI*AVAST Software            311009012882   
3             ZA  DRI*AVAST Software              1156962062   
4             ZA  DRI*AVAST Software              1411163460   

     mid_entity_code new_status  order_entity_code payment_amount  \
0  DR_IRELAND-ENTITY  Completed  DR_IRELAND-ENTITY         120.74   
1  DR_IRELAND-ENTITY   Declined  DR_IRELAND-ENTITY          89.99   
2      DR_INC-ENTITY  Completed      DR_INC-ENTITY          49.99   
3  DR_IRELAND-ENTITY   Declined  DR_IRELAND-ENTITY            499   
4  DR_IRELAND-ENTITY   Declined  DR_IRELAND-ENTITY            499   

   payment_amount_usd payment_currency payment_method_id payment_service_id  \
0           80.981339              NZD              Visa        netgiro-bms   
1           66.222603              AUD        MasterCard                nab   
2           49.990002              USD              Visa          firstdata   
3           34.959465              ZAR              Visa        netgiro-seb   
4           34.959465              ZAR              Visa        netgiro-bms  

In [33]:
"""Show all columns"""
import pandas as pd
from IPython.display import display

pd.options.display.max_columns = None
display(pdf_all_bin_profile)

Unnamed: 0 received_date   site_id  subscription_id  transaction_id  \
0                 0    2018-08-21       acd      10418939801      7242559663   
1                 1    2018-08-21       acd      10465808701      7242603393   
2                 2    2018-08-21       acd      10467977001      7242614223   
3                 3    2018-08-21       acd       2852088109      7242339943   
4                 4    2018-08-21       acd       2852088109      7242340063   
5                 5    2018-08-21       acd       2852088209      7242338563   
6                 6    2018-08-21       acd       2852088209      7242338603   
7                 7    2018-08-21       acd       3524602801      7242374343   
8                 8    2018-08-21       acd       3524791701      7242381883   
9                 9    2018-08-21       acd       3525235101      7242380573   
10               10    2018-08-21       acd       3525499801      7242603743   
11               11    2018-08-21       acd       9218770801      7242442593   
12               12    2018-08-21       acd       9219793901      7242628003   
13               13    2018-08-21       acd       9220131101      7242464823   
14               14    2018-08-21   acronis      11573730601      7243075563   
15               15    2018-08-21   acronis      11585824701      7243057603   
16               16    2018-08-21   acronis      11585844201      7243098303   
17               17    2018-08-21   acronis      11586253301      7243086663   
18               18    2018-08-21   acronis      11712014401      7243108313   
19               19    2018-08-21   acronis      11863083401      7242352553   
20               20    2018-08-21   acronis      11863083401      7242352623   
21               21    2018-08-21   acronis      11879276701      7243118603   
22               22    2018-08-21   acronis      11880629801      7243218383   
23               23    2018-08-21   acronis      11881166501      7243185043   
24               24    2018-08-21   acronis      11881701401      7243239933   
25               25    2018-08-21   acronis      11882183301      7243166183   
26               26    2018-08-21   acronis      11882940301      7243212073   
27               27    2018-08-21   acronis      11883262101      7243212433   
28               28    2018-08-21   acronis      11980427801      7243422613   
29               29    2018-08-21   adbecnn      10007376301      7242537543   
...             ...           ...       ...              ...             ...   
4281755     4281755    2018-09-21  trendoem        744668009      7411584613   
4281756     4281756    2018-09-21  trendoem        744726909      7411584623   
4281757     4281757    2018-09-21   trendsb      10550864001      7411808503   
4281758     4281758    2018-09-21   trendsb      10550864001      7411808913   
4281759     4281759    2018-09-21   trendsb      10598706901      7411943693   
4281760     4281760    2018-09-21   trendsb      10598980801      7411847083   
4281761     4281761    2018-09-21   trendsb      10598980801      7411847133   
4281762     4281762    2018-09-21   trendsb      10600705901      7411847453   
4281763     4281763    2018-09-21   trendsb       3590408601      7411608113   
4281764     4281764    2018-09-21   trendsb       3614518801      7411882833   
4281765     4281765    2018-09-21   trendsb       3623965701      7411608593   
4281766     4281766    2018-09-21   trendsb       9277232501      7411648013   
4281767     4281767    2018-09-21   trendsb       9291649901      7411844463   
4281768     4281768    2018-09-21   trendsb       9291649901      7411844603   
4281769     4281769    2018-09-21   trendsb       9291674201      7411660603   
4281770     4281770    2018-09-21   trendsb       9291674201      7411660723   
4281771     4281771    2018-09-21   trendsb       9312410601      7411650183   
4281772     4281772    2018-09-21   trendsb       9312410601      7411650353   
4281773  

In [72]:
# pdf_debit = pdf_f[pdf_f['funding_source'] == 'Debit']
# pdf_debit.to_csv(WORK_DIR + 'TOD_Debit_Avast_TM_Kasper.csv')
# pdf_debit.shape   

pdf_debit = pdf_all_bin_profile_with_count[pdf_all_bin_profile_with_count['funding_source'] == 'Debit']
pdf_debit.to_csv(WORK_DIR + 'TOD_Debit_Avast_TM_Kasper.csv')
pdf_debit.shape   



(383066, 52)

In [71]:
pdf_debit.head()

received_date site_id subscription_id transaction_id    transaction_date  \
2     2018-08-23   avast     10021676301     7258325943 2018-08-23 16:15:39   
9     2018-08-23   avast     10031628001     7254482713 2018-08-23 04:45:33   
10    2018-08-23   avast     10036725801     7254313803 2018-08-23 02:45:55   
14    2018-08-23   avast     10054995101     7258228043 2018-08-23 14:45:47   
15    2018-08-23   avast     10054995101     7258228263 2018-08-23 14:45:48   

   added_expiry_years         bank_code  \
2                      847B99784061F302   
9                                    RB   
10                     16B2EEC78A2D3D8C   
14                  4            NON3DS   
15                  4            NON3DS   

                                 bank_name billing_country     bin  \
2    Bank of America, National Association              US  481583   
9                     CREDIT AGRICOLE S.A.              FR  513106   
10  Wells Fargo Bank, National Association              US  434258   
14                         Lloyds Bank Plc              GB  492181   
15                         Lloyds Bank Plc              GB  492181   

    card_brand                         card_category card_class  \
2         Visa                          Visa Classic   Consumer   
9   MasterCard  Gold MasterCard Card Immediate Debit   Consumer   
10        Visa                          Visa Classic   Consumer   
14        Visa                          Visa Classic   Consumer   
15        Visa                          Visa Classic   Consumer   

                                card_usage cc_expiration_date  \
2         Debit Hybrid (PIN and Signature)               0719   
9   NON USA Consumer Debit - No Pin Access               0119   
10        Debit Hybrid (PIN and Signature)               0619   
14  NON USA Consumer Debit - No Pin Access               1021   
15  NON USA Consumer Debit - No Pin Access               1021   

                 cid date_increment  day_of_month day_of_week  \
2                                              23    thursday   
9                                              23    thursday   
10                                             23    thursday   
14                                4            23    thursday   
15  RETRY_DECLINED.2              4            23    thursday   

                              event_uuid exp_setting_id funding_source  \
2   203cb185-c4e1-48e4-bcd4-9a5bdb8b8def                         Debit   
9   6f5025f0-52fd-4451-8b9f-65f8ab8ee405                         Debit   
10  13010b22-ef46-4e58-8900-c8c2b8e444d8                         Debit   
14  dca4915d-84b3-4418-bbd9-a15469461837        3836591          Debit   
15  882dd75a-2ab8-4a54-9956-ee86428ce88f        3836591          Debit   

   issuer_country merchant_descriptor merchant_number    mid_entity_code  \
2              US  DRI*AVAST Software    311009012882      DR_INC-ENTITY   
9              FR  DRI*AVAST Software      1785390597  DR_IRELAND-ENTITY   
10             US  DRI*AVAST Software    311009012882      DR_INC-ENTITY   
14             GB  DRI*AVAST Software      1850961065  DR_IRELAND-ENTITY   
15             GB  DRI*AVAST Software      1156962062  DR_IRELAND-ENTITY   

   new_status  order_entity_code payment_amount  payment_amount_usd  \
2   Completed      DR_INC-ENTITY          49.99           49.990002   
9   Completed  DR_IRELAND-ENTITY           7.99            9.281184   
10  Completed      DR_INC-ENTITY           8.65            8.650000   
14   Declined  DR_IRELAND-ENTITY          49.99           64.572083   
15   Declined  DR_IRELAND-ENTITY          49.99           64.572083   

   payment_currency payment_method_id payment_service_id renew_att_num  \
2               USD              Visa          firstdata             1   
9               EUR        MasterCard        netgiro-bnp             1   
10              USD              Visa          firstdata             1   
14              GBP              Visa 

In [69]:
p = pdf_all_bin_profile_with_count[pdf_all_bin_profile_with_count['renew_att_num'] == '1']
p.to_csv(WORK_DIR + 'TOD_ALL_First_Attempt.csv')
p.shape

(608333, 52)

In [70]:
p = pdf_all_bin_profile_with_count[pdf_all_bin_profile_with_count['renew_att_num'] == '1']
p = p[p['issuer_country'] == 'US']
p.to_csv(WORK_DIR + 'TOD_ALL_First_Attempt_Issuer_US.csv')
p.shape

(222572, 52)

In [68]:
p = pdf_debit[pdf_debit['renew_att_num'] == '1']
p.to_csv(WORK_DIR + 'TOD_Debit_First_Attempt.csv')
p.shape

(211262, 52)

In [63]:
p = pdf_all_bin_profile_with_count[pdf_all_bin_profile_with_count['issuer_country'] == 'US']
p.to_csv(WORK_DIR + 'TOD_ALL_Issuer_US.csv')
p.shape

(386338, 52)

In [57]:
p = pdf_debit[pdf_debit['issuer_country'] == 'US']
p.to_csv(WORK_DIR + 'TOD_DEBIT_Issuer_US.csv')
p.shape

(151699, 52)

In [58]:
p = pdf_debit[pdf_debit['issuer_country'] == 'GB']
p.to_csv(WORK_DIR + 'TOD_DEBIT_Issuer_GB.csv')
p.shape

(38731, 52)

In [59]:
p = pdf_all_bin_profile_with_count[pdf_all_bin_profile_with_count['issuer_country'] == 'US']
p.to_csv(WORK_DIR + 'TOD_ALL_Issuer_US.csv')
p.shape

(386338, 52)

In [60]:
p = pdf_all_bin_profile_with_count[pdf_all_bin_profile_with_count['issuer_country'] == 'GB']
p.to_csv(WORK_DIR + 'TOD_ALL_Issuer_GB.csv')
p.shape

(64203, 52)

In [61]:
pdf_all_bin_profile_with_count.to_csv(WORK_DIR + 'TOD_ALL_Avast_TM_Kasper.csv')

In [13]:
pdf_f = pd.read_csv(WORK_DIR + 'TOD_ALL_Avast_TM_Kasper.csv')

/opt/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [62]:
pdf_debit['site_id'].value_counts()

avast       168329
avgstore    152633
kasperus     26395
kasper       11227
tmamer        9324
kasperuk      5486
tmapac        4335
kasperla      1486
tmecon        1356
tmemea        1097
kaspersk      1094
kasperde       281
avastbr         22
avgbr            1
Name: site_id, dtype: int64

In [4]:
pdf_all.to_csv(WORK_DIR + 'TimeOfDay_raw_data.csv')

In [5]:
pdf_all = pd.read_csv(WORK_DIR + 'TimeOfDay_raw_data.csv')

/opt/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (15,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
pdf_all['received_date'].value_counts(sort=True).sort_index()

2018-08-12     82425
2018-08-13     81997
2018-08-14     81675
2018-08-15     84389
2018-08-16     87677
2018-08-17     85824
2018-08-18     86535
2018-08-19     84281
2018-08-20     84680
2018-08-21     83613
2018-08-22     84643
2018-08-23     86622
2018-08-24     94221
2018-08-25     86824
2018-08-26     84720
2018-08-27     83157
2018-08-28    115703
2018-08-29     68163
2018-08-30     70471
2018-08-31     67264
2018-09-01     86383
2018-09-02     86448
2018-09-03     85010
2018-09-04     90056
2018-09-05     82683
2018-09-06     83704
2018-09-07     83700
2018-09-08     84108
2018-09-09     85663
2018-09-10     85436
2018-09-11     88576
2018-09-12     84053
2018-09-13     85101
2018-09-14     85062
2018-09-15     90568
2018-09-16     89998
2018-09-17     87670
2018-09-18     87913
2018-09-19     85606
2018-09-20     86526
2018-09-21     85609
2018-09-22     87597
2018-09-23     88483
2018-09-24     90858
2018-09-25     82856
2018-09-26     84641
2018-09-27     89168
2018-09-28   

In [5]:
import pandas as pd
WORK_DIR = '/var/spark/ml_files/'
pdf_f = pd.read_csv(WORK_DIR + 'TOD_Debit_Avast_TM_Kasper.csv')

In [73]:
df_decline_type = pd.read_csv(WORK_DIR + 'Decline_Type.csv')

In [76]:
pdf_debit.shape

(383066, 52)

In [88]:
def is_fraud(response_msg):
    '''Converts to decline_type based on the given response_msg'''
    dec_type = df_decline_type[df_decline_type['DECLINE_TEXT'] == response_msg]['DECLINE_TYPE']
    if not dec_type.empty:
        dec_str = dec_type.iloc[0]
        if dec_str == 'Hard Fraud':
            return 1
    
    return 0


pdf_all_bin_profile_with_count['is_fraud'] = pdf_all_bin_profile_with_count['response_message'].apply(is_fraud) 

In [90]:
drop_columns = ['transaction_id', 'subscription_id', 'cid', 'cc_expiration_date', 'event_uuid', 'exp_setting_id' ]
pdf_all_bin_profile_with_count = pdf_all_bin_profile_with_count.drop(drop_columns, axis=1)

In [91]:
pdf_all_bin_profile_with_count.to_csv(WORK_DIR + 'TOD_ALL_is_fraud.csv')

In [97]:
# pdf_debit['is_fraud'].value_counts()
p = pdf_all_bin_profile_with_count[pdf_all_bin_profile_with_count['issuer_country'] == 'US']

gp = p.groupby(['transaction_hour'])
df_gp = gp.agg({'is_fraud':['sum', 'count']})
df_gp['is_fraud_rate'] = df_gp[('is_fraud', 'sum')] / df_gp[('is_fraud', 'count')]
txn_hour_gp = df_gp[df_gp[('is_fraud', 'count')] > 5]

txn_hour_gp.sort_values(by=['is_fraud_rate',('is_fraud', 'count')], ascending=False)

is_fraud        is_fraud_rate
                      sum  count              
transaction_hour                              
3                    1150  25830      0.044522
9                    1151  26138      0.044036
8                    1168  26878      0.043456
4                     985  23115      0.042613
7                    1005  24887      0.040383
2                    1073  26603      0.040334
10                    809  20280      0.039892
5                     845  21517      0.039271
6                    1001  25560      0.039163
14                    915  23376      0.039143
13                    775  19930      0.038886
15                   1021  26321      0.038790
12                    763  19926      0.038292
11                    690  18853      0.036599
16                   1460  41693      0.035018
0                      43   1588      0.027078
1                     301  13841      0.021747

In [94]:
# pdf_debit['is_fraud'].value_counts()
p = pdf_debit[pdf_debit['issuer_country'] == 'US']

gp = p.groupby(['transaction_hour'])
df_gp = gp.agg({'is_fraud':['sum', 'count']})
df_gp['is_fraud_rate'] = df_gp[('is_fraud', 'sum')] / df_gp[('is_fraud', 'count')]
txn_hour_gp = df_gp[df_gp[('is_fraud', 'count')] > 5]

txn_hour_gp.sort_values(by=['is_fraud_rate',('is_fraud', 'count')], ascending=False)

is_fraud        is_fraud_rate
                      sum  count              
transaction_hour                              
3                     793  10710      0.074043
9                     643   8989      0.071532
4                     690   9730      0.070915
15                    699   9955      0.070216
13                    514   7512      0.068424
8                     632   9260      0.068251
2                     724  10628      0.068122
14                    601   8836      0.068017
7                     695  10233      0.067918
10                    572   8569      0.066752
6                     694  10518      0.065982
12                    504   7806      0.064566
5                     575   8996      0.063917
16                    967  15359      0.062960
11                    446   7089      0.062914
0                      27    681      0.039648
1                     224   6827      0.032811

In [18]:
# pdf_f['funding_source'].value_counts()
# pdf_f[pdf_f['funding_source'] == 'Debit']['success'].value_counts()
pdf_debit['success'].value_counts()



0    584481
1    227224
Name: success, dtype: int64

In [8]:
# txn_count_all.to_csv(WORK_DIR + 'txn_count.csv')
# pdf_all_bin_profile.to_csv(WORK_DIR + 'tod_all_bin_profile.csv')

In [10]:
txn_count_all = pd.read_csv( WORK_DIR +'txn_count.csv')

In [14]:
pdf_all_bin_profile_with_count[['transaction_hour', 'payment_service_id','processor_count','total_count']].head(5)

transaction_hour payment_service_id  processor_count  total_count
0                 2              litle            37270       654456
1                 2                nab            30807       654456
2                 2        netgiro-bms           222325       654456
3                 0                mes            25290       663703
4                 0          firstdata            31962       663703

In [15]:
# pdf_f_with_count.to_csv(WORK_DIR + 'TOD_Debit_Avast_TM_Kasper_with_Count.csv')
pdf_all_bin_profile_with_count.to_csv(WORK_DIR + 'TOD_ALL_Avast_TM_Kasper_with_Count.csv')

In [18]:
# txn_count_all.shape
pdf_debit = pd.read_csv(WORK_DIR + 'TOD_Debit_Avast_TM_Kasper.csv').drop(['Unnamed: 0'], axis=1)

In [87]:
pdf_debit.to_csv(WORK_DIR + 'TOD_Debit_Avast_TM_Kasper_is_fraud.csv')

In [35]:
gp = pdf_all.groupby(['transaction_hour'])
df_gp = gp.agg({'success':['sum', 'count']})
df_gp['successful_rate'] = df_gp[('success', 'sum')] / df_gp[('success', 'count')]
txn_hour_gp = df_gp[df_gp[('success', 'count')] > 5]

txn_hour_gp.sort_values(by=['successful_rate',('success', 'count')], ascending=False)

# pdf_all.groupby('transaction_hour').agg(['count'])

success         successful_rate
                     sum   count                
transaction_hour                                
0                 105056  163170        0.643844
1                 216825  381331        0.568601
13                  6326   15543        0.407000
15                  7508   18589        0.403895
14                  6958   17646        0.394310
11                  7674   19674        0.390058
12                  8707   22421        0.388341
10                  7557   20285        0.372541
9                  10951   31014        0.353099
3                   9457   27201        0.347671
2                  68560  199223        0.344137
4                   8655   25212        0.343289
5                  10233   29835        0.342986
8                   9591   27976        0.342830
7                  10441   30737        0.339688
6                   9727   29029        0.335079
16                 11848   35741        0.331496

In [34]:
group = 'transaction_hour'
df_plot = txn_hour_gp['successful_rate'].plot(kind="linbe", figsize=(15, 4), xticks = range(24))
df_plot.set(xlabel=group, ylabel="success rate")

[Text(0,0.5,'success rate'), Text(0.5,0,'transaction_hour')]

In [6]:
pdf_all = pd.read_csv(WORK_DIR + 'TimeOfDay_raw_data.csv')

In [11]:
pdf_all[pdf_all['renew_att_num'] == 5]

Unnamed: 0 received_date   site_id  subscription_id  transaction_id  \
97               97    2018-09-15  avgstore      10559980501      7379231503   
246             246    2018-09-15  avgstore      12163854101      7382062873   
256             256    2018-09-15   adbehkr      10216797001      7379161403   
265             265    2018-09-15   adbehkr      11262629501      7379451203   
298             298    2018-09-15   adbehap      12001881301      7379572513   
310             310    2018-09-15  avgstore      12164050501      7379697023   
316             316    2018-09-15   adbehap      12001345501      7379566213   
361             361    2018-09-15  avgstore      12163941401      7379680913   
464             464    2018-09-15   adbehap      11998719901      7379624003   
698             698    2018-09-15   adbehap      11997697701      7379631343   
808             808    2018-09-15  avgstore      12163742901      7380334493   
893             893    2018-09-15   adbehkr      11419065601      7379485923   
952             952    2018-09-15   adbehme      11864304901      7378971523   
1008           1008    2018-09-15  avgstore      10559826801      7380336003   
1082           1082    2018-09-15  avgstore      12163888901      7380422073   
1084           1084    2018-09-15   adbehkr      11863982301      7378971573   
1093           1093    2018-09-15   adbehkr      12001530601      7379639783   
1109           1109    2018-09-15   adbehap       3899016401      7379013493   
1140           1140    2018-09-15  avgstore      12164127801      7380418213   
1172           1172    2018-09-15   adbehap      12001711601      7379560343   
1268           1268    2018-09-15   adbehap      11998138701      7379632583   
1335           1335    2018-09-15  avgstore      12163716201      7379716953   
1416           1416    2018-09-15   adbehap      12000474201      7379595093   
1418           1418    2018-09-15   adbehkr      10952045201      7379410873   
1428           1428    2018-09-15  avgstore      12163807501      7379699393   
1483           1483    2018-09-15   adbehru      10952980001      7379413133   
1550           1550    2018-09-15   adbehkr      10214999601      7379160463   
1593           1593    2018-09-15   adbeheu      11571317801      7379503913   
1633           1633    2018-09-15   adbehil       3513902601      7378985193   
1653           1653    2018-09-15   adbehme      10952080701      7379410053   
...             ...           ...       ...              ...             ...   
1140153      284101    2018-09-15   adbehkr      11259994201      7379460053   
1140154      284102    2018-09-15  avgstore      10533366001      7379216673   
1140159      284107    2018-09-15   adbehme      11259526801      7379447783   
1140185      284133    2018-09-15  avgstore      12163801501      7383397713   
1140190      284138    2018-09-15  avgstore      12163732501      7383510203   
1140303      284251    2018-09-15  avgstore      12164141901      7379704063   
1140308      284256    2018-09-15   adbehme      11863239701      7378971293   
1140468      284416    2018-09-15   adbehap      11999413901      7379601103   
1140482      284430    2018-09-15   adbehkr      11998150301      7379635113   
1140519      284467    2018-09-15   adbehap       9302635801      7379346083   
1140555      284503    2018-09-15   adbehkr      10952372201      7379411993   
1140672      284620    2018-09-15  avgstore      12164086101      7379697233   
1140685      284633    2018-09-15   adbeheu      11998992801      7379601523   
1140723      284671    2018-09-15   adbehme      10954448701      7379411833   
1140731      284679    2018-09-15  avgstore      12145613801      7379688043   
1140821      284769    2018-09-15   adbehme      11105869701      7379436583   
1140824      284772    2018-09-15   adbehru       5502290201      7379027413   
1140825      284773    2018-09-15   adbehbr      10216592301      7379166123   
1140837  

In [14]:
pdf_all[pdf_all['subsegment_id'] == 16786048800]

Unnamed: 0 received_date  site_id  subscription_id  transaction_id  \
1084        1084    2018-09-15  adbehkr      11863982301      7378971573   

         transaction_date  added_expiry_years bank_code  \
1084  2018-09-15 05:30:07                 NaN    NON3DS   

                   bank_name billing_country       bin card_brand  \
1084  Shinhan Card Co., Ltd.              KR  436420.0       Visa   

     card_category card_class                                      card_usage  \
1084  Visa Classic   Consumer  Debit - NON USA Consumer Debit - No Pin Access   

     cc_expiration_date  cid  date_increment  day_of_month day_of_week  \
1084               0521  NaN             NaN            15    saturday   

                                event_uuid  exp_setting_id funding_source  \
1084  b3f921bc-87a9-4f53-bbff-8cd43966f1e0             NaN          Debit   

     issuer_country merchant_descriptor merchant_number    mid_entity_code  \
1084             KR   DRI*Adobe Systems      1156962062  DR_IRELAND-ENTITY   

     new_status order_entity_code  payment_amount  payment_amount_usd  \
1084  Completed   DR_KOREA-ENTITY         11000.0                 9.8   

     payment_currency payment_method_id payment_service_id  renew_att_num  \
1084              KRW              Visa        netgiro-seb              5   

      request_amount  request_amount_usd request_currency  requisition_id  \
1084         11000.0                 9.8              KRW     52051653500   

      response_code            response_message status  subsegment_id  \
1084            0.0  SUCCESS[00] [342960] [] []    New    16786048800   

     transaction_date_in_string  transaction_hour transaction_type  success  
1084        2018-09-15 00:30:07                 0        Authorize        1

In [7]:
import time

import pandas as pd


from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit, lower
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import pyspark_cassandra
        
import calendar, datetime
from datetime import timedelta

WORK_DIR = '/var/spark/ml_files/'

def convert_status(value):
    if value == 'Completed' or value == 'Reversed':
        return 1
    else:
        return 0

def _map_to_pandas(rdds):
    """ Needs to be here due to pickling issues """
    return [pd.DataFrame(list(rdds))]

def toPandas(df, n_partitions=None):
    """
    Returns the contents of `df` as a local `pandas.DataFrame` in a speedy fashion. The DataFrame is
    repartitioned if `n_partitions` is passed.
    :param df:              pyspark.sql.DataFrame
    :param n_partitions:    int or None
    :return:                pandas.DataFrame
    """
    if n_partitions is not None: df = df.repartition(n_partitions)
    df_pand = df.rdd.mapPartitions(_map_to_pandas).collect()
    df_pand = pd.concat(df_pand)
    df_pand.columns = df.columns
    return df_pand
    
udfValueToStatus = udf(convert_status, IntegerType())  
createDDL = """CREATE OR REPLACE TEMPORARY VIEW cpg_transaction
     USING org.apache.spark.sql.cassandra
     OPTIONS (
     table "cpg_transaction",
     keyspace "subs",
     cluster "MLP Cluster",
     pushdown "true")"""
spark.sql(createDDL)

# df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date > '2018-08-12' ")
df = spark.sql("SELECT * FROM cpg_transaction WHERE received_date > '20180101' and received_date < '20180131'")

sdf =  df\
    .filter("!(renew_att_num == '1' and new_status in ('Completed','Reversed') and cid is null and status == 'New')")\
    .withColumn("success", udfValueToStatus("new_status"))\
    .withColumn("day_of_month", df["day_of_month"].cast(IntegerType()))\
    .withColumn("transaction_hour", df["transaction_hour"].cast(IntegerType()))\
    .withColumn("day_of_week", lower(col("day_of_week")))\
     .cache()

print(sdf.schema)

pdf_all = sdf.toPandas()
# pdf_all = toPandas(sdf, 4)  
pdf_all.shape

StructType(List(StructField(received_date,StringType,true),StructField(site_id,StringType,true),StructField(subscription_id,StringType,true),StructField(event_uuid,StringType,true),StructField(added_expiry_years,StringType,true),StructField(bank_code,StringType,true),StructField(bank_name,StringType,true),StructField(billing_country,StringType,true),StructField(bin,StringType,true),StructField(card_brand,StringType,true),StructField(card_category,StringType,true),StructField(card_class,StringType,true),StructField(card_usage,StringType,true),StructField(cc_expiration_date,StringType,true),StructField(cid,StringType,true),StructField(date_increment,StringType,true),StructField(day_of_month,IntegerType,true),StructField(day_of_week,StringType,true),StructField(exp_setting_id,StringType,true),StructField(funding_source,StringType,true),StructField(issuer_country,StringType,true),StructField(merchant_descriptor,StringType,true),StructField(merchant_number,StringType,true),StructField(mid_e

(1088927, 48)

In [8]:
pdf_all.to_csv(WORK_DIR + 'raw_data_01.csv')

In [9]:
"""Show all columns"""
import pandas as pd
from IPython.display import display

pd.options.display.max_columns = None

pdf_all[(pdf_all['site_id'] == 'avast') & (pdf_all['renew_att_num'] == '3') & (pdf_all['success'] == 1)]

received_date site_id subscription_id  \
10858        20180122   avast      1913859009   
10991        20180122   avast      2442557509   
11001        20180122   avast      2443114609   
11022        20180122   avast      2443429709   
11376        20180122   avast      3027160009   
11468        20180122   avast      3142802009   
11481        20180122   avast      3143097309   
11601        20180122   avast      3145629509   
11668        20180122   avast      3155682809   
11702        20180122   avast      3155896409   
11721        20180122   avast      3156271809   
11803        20180122   avast      3156771309   
11828        20180122   avast      3156868609   
11844        20180122   avast      3156967309   
11868        20180122   avast      3157070809   
12010        20180122   avast      3157610709   
12034        20180122   avast      3157731909   
12070        20180122   avast      3157867309   
13501        20180122   avast      3906231301   
13512        20180122   avast      3906546701   
13523        20180122   avast      3906582301   
13634        20180122   avast      3906988601   
13759        20180122   avast      3907262801   
13778        20180122   avast      3907310101   
13801        20180122   avast      3907360701   
13811        20180122   avast      3907364401   
13913        20180122   avast      3907552901   
14212        20180122   avast      3908041101   
14282        20180122   avast      3908182001   
14396        20180122   avast      3908356501   
...               ...     ...             ...   
1037982      20180109   avast      9658455601   
1037995      20180109   avast      9658494601   
1038028      20180109   avast      9658561501   
1038060      20180109   avast      9658644801   
1038111      20180109   avast      9658742401   
1038123      20180109   avast      9658758001   
1038162      20180109   avast      9658844301   
1038292      20180109   avast      9659068101   
1038341      20180109   avast      9659138701   
1038342      20180109   avast      9659142101   
1038356      20180109   avast      9659180501   
1038361      20180109   avast      9659183501   
1038364      20180109   avast      9659191101   
1038396      20180109   avast      9659232201   
1038411      20180109   avast      9659271201   
1038554      20180109   avast      9659513501   
1038630      20180109   avast      9659643301   
1038728      20180109   avast      9659796101   
1038736      20180109   avast      9659816101   
1038847      20180109   avast      9659981601   
1038935      20180109   avast      9660126201   
1038969      20180109   avast      9660200301   
1038970      20180109   avast      9660202101   
1039044      20180109   avast      9660315901   
1039100      20180109   avast      9660408101   
1039105      20180109   avast      9660419701   
1039135      20180109   avast      9660464301   
1039155      20180109   avast      9660511101   
1066155      20180111   avast      2425502809   
1067832      20180111   avast      3880220101   

                                   event_uuid added_expiry_years  \
10858    e35bd044-212a-44c4-8928-ae3d17eca989               None   
10991    56ea8de9-1db2-492b-88c9-8e8a04d85fcc               None   
11001    80c8c5e3-2bd6-4c52-b757-f3d7b6a0128e               None   
11022    1e58612f-0e32-40df-954a-ebc62fd96a08               None   
11376    42a90087-5037-4db9-80f3-263e49e1b2ab               None   
11468    168f4329-dd33-47aa-9814-030afc2b1eb8               None   
11481    0a6e9f2a-7de7-4ea7-9ebf-afb9419c6e40               None   
11601    b5649a5b-74c2-4f90-b0bb-d55610f23f52               None   
11668    c7d3a5ad-9e1e-46e5-8e03-1bace71e8a9c               None   
11702    ea2bd718-43f3-4a93-be45-e3b8eac1e6ff               None   
11721    0c9ed870-18f9-4e14-961d-f40769dc71f5               None   
11803    d9cd2214-6add-4ac3-8c13-2915b68bc1f6               None   
11828    21e1fffe-64c0-4558-ab71-f49009982e17               None   
11844    0203d950